<a href="https://www.kaggle.com/code/pyliraghuram/xgb-regression-1?scriptVersionId=144445752" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

# train test assingn

In [ ]:
train =train.drop(['Id','Street', 'Utilities','LandSlope','Condition2','RoofMatl','CentralAir','LowQualFinSF','3SsnPorch','ScreenPorch','PoolArea','MiscVal'], axis=1)
test =test.drop(['Id','Street', 'Utilities','LandSlope','Condition2','RoofMatl','CentralAir','LowQualFinSF','3SsnPorch','ScreenPorch','PoolArea','MiscVal'], axis=1)
for df in (train, test):
    df["n_bathrooms"] = df["BsmtFullBath"] + (df["BsmtHalfBath"]*0.5) + df["FullBath"] + (df["HalfBath"]*0.5)
    df["area_with_basement"]  = df["GrLivArea"] + df["TotalBsmtSF"]



In [ ]:
X_train = train.select_dtypes(include=['number']).copy()
X_train = X_train.drop(['SalePrice'], axis=1)
y_train = train["SalePrice"]
X_test  = test.select_dtypes(include=['number']).copy()
 

In [ ]:
#sns.pairplot(train)

# Feature Scaling

In [ ]:
import xgboost as xg
from sklearn.preprocessing import RobustScaler


rrs =RobustScaler()
X_train= rrs.fit_transform(X_train)
X_test = rrs.transform(X_test)



In [ ]:
import warnings
warnings.filterwarnings("ignore")

# implementing XGB using Gridsearch CV

In [ ]:
xgb = xg.XGBRegressor(eval_metric='rmsle')
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(xgb,scoring = 'accuracy',cv = 5,param_grid  = {'max_depth' : [3,5,8],"n_estimators": [500, 600, 700],"learning_rate": [0.01,0.015],
'gamma': [0,0.2],'reg_lambda': [0,1.0]})

# fit the model

gs.fit(X_train, y_train)

print("Best Parameters:", gs.best_params_)
print("Best Score:", gs.best_score_)

In [ ]:
xgb = xg.XGBRegressor(gamma = 0,learning_rate = 0.01,max_depth = 3,n_estimators = 500,reg_lambda = 0)
xgb.fit(X_train,y_train)
#print(xgb.score(X_train,y_train))

In [ ]:
prediction = xgb.predict(X_test)

In [ ]:
solution = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
y_test = solution['SalePrice']


In [ ]:
from sklearn.metrics import mean_squared_log_error
RMSLE = np.sqrt( mean_squared_log_error(y_test, prediction) )
print("The score is %.5f" % RMSLE )

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
print("Mean Absolute Error:", mean_absolute_error(y_test, prediction))
print("Root Mean Squared Error:", np.sqrt(mean_absolute_error(y_test, prediction)))